## 一
<!-- * PCA  -->
bertopic 主题聚类 $\to$ 因子分析 公因子
* 因子分析（可解释） 5、6个
* 4w $\times$ p matrix
## 分类模型
smote采样/没有采样的
    baseline model
  * 逻辑回归
  * svc
  * DTC
  * RF
    跑一下baseline模型

    进阶模型（我们提出的模型）
  * CNN
共10个模型
## 评价准则
精确率 召回率
* CNN 采样/未采样/sharp value可解释性（可能是较好的）

In [8]:
# 读入数据
import pandas as pd
rd=pd.read_csv('E:\code\python_code\BDManager\sunshaolong\model\因子分析20维数据.csv',index_col=0)
rd.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,label
0,0.078881,0.109237,0.121366,0.081104,0.085179,1.525199,0.016275,0.019840,-0.017074,0.117229,...,0.401077,0.244692,-0.423489,-0.252544,-0.104657,0.061354,0.212366,-0.106895,-0.174594,1
1,1.129565,1.324284,1.503223,1.182611,1.149371,20.243247,0.036048,0.587170,0.579361,1.030450,...,3.122662,1.930230,-2.236521,-1.629814,-0.415996,-0.191336,0.942498,-0.306906,-0.766555,0
2,0.113380,0.137732,0.187015,0.117512,0.129966,0.981701,0.141062,0.040679,0.001235,0.060275,...,0.656048,0.285357,-0.509809,-0.417252,-0.088661,-0.030604,0.214359,0.052981,-0.053427,1
3,0.130303,0.154464,0.197254,0.134266,0.144115,0.983246,0.144722,0.047023,0.004954,0.020829,...,0.657501,0.272685,-0.516043,-0.421925,-0.081141,-0.025779,0.218346,0.061396,-0.057581,1
4,0.117511,0.128078,0.254737,0.128391,0.151780,0.334925,0.328880,0.037059,-0.013657,0.211102,...,0.940115,0.429599,-0.581791,-0.879451,-0.135077,-0.183515,0.245517,0.520213,0.172435,1


In [9]:
X=rd.iloc[:,:-1]
y=rd.iloc[:,-1]

In [192]:
y.value_counts()

1    41493
0     1649
Name: label, dtype: int64

In [189]:
# pd.concat([X,y],axis=1).to_csv('E:\code\python_code\BDManager\sunshaolong\model\因子分析20维数据.csv')

In [51]:
rd['label'].value_counts()

 1    41493
-1     1649
Name: label, dtype: int64

In [3]:
# 划分训练与测试集
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0,test_size=0.2)

ModuleNotFoundError: No module named 'sklearn'

# 1. 逻辑回归

In [70]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
pip=Pipeline(
    [
        ('std',StandardScaler()),
        ('logit',LogisticRegression())
    ]
)
# logit=LogisticRegression()
pip.fit(X_train,y_train)

E:\Anoconda\Lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('std', StandardScaler()), ('logit', LogisticRegression())])

In [71]:
pip.score(X_test,y_test)

0.9623363077992815

In [72]:
y_test.value_counts()

 1    8311
-1     318
Name: label, dtype: int64

In [4]:
X_test_neg=X.loc[y_test[y_test==-1].index]

In [74]:
pip.score(X_test_neg,y_test[y_test==-1])

0.018867924528301886

可以看出未采样条件下对于负类的识别正确率仅为1%，样本极度不平衡的情况也导致了正确率虚高。

# 2.支持向量分类器（SVC）

In [75]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
pip=Pipeline(
    [
        ('std',StandardScaler()),
        ('svc',SVC())
    ]
)
# logit=LogisticRegression()
for i in range(1,4):
    pip.set_params(svc__degree=i)
    pip.fit(X_train,y_train)
    print('=================degree:',i,'=================')
    print('测试集上的得分：',pip.score(X_test,y_test))
    print('测试集上负例的得分：',pip.score(X_test_neg,y_test[y_test==-1]))

KeyboardInterrupt: 

# 3.Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier()
dt.fit(X_train,y_train)
print('测试集上的得分：',dt.score(X_test,y_test))
print('测试集上负例的得分：',dt.score(X_test_neg,y_test[y_test==-1]))


测试集上的得分： 0.9457642832309654
测试集上负例的得分： 0.31446540880503143


# 4.RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()
rfc.fit(X_train,y_train)
print('测试集上的得分：',rfc.score(X_test,y_test))
print('测试集上负例的得分：',rfc.score(X_test_neg,y_test[y_test==-1]))

测试集上的得分： 0.9634951906362267
测试集上负例的得分： 0.1949685534591195


# 5.CNN


In [7]:
import torch as t
import torch.nn as nn
import torch.nn.functional as F
# F.max_pool2d()

In [9]:
import torch.nn as nn
import torch.nn.functional as F

class  Net(nn.Module):
    def __init__(self):
        # nn.Module子类的函数必须在构造函数中执行父类的构造函数
        # 下式等价于nn.Module.__init__(self)
        super(Net, self).__init__() # 继承Net子类的超类nn.Module
        # 卷积层 1表示输入为单通道，6表示输出通道数，5表示卷积核为5*5
        self.conv1 = nn.Conv2d(1, 6, 5)
        # 卷积层
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 仿射层/全联接层/线性层
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        #卷积—>激活—>池化
        x=F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x=F.max_pool2d(F.relu(self.conv2(x)), 2)
        # reshape ,’-1’表示自适应
        x = x.view(x.size()[0],-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print(net) 

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [15]:
b=t.arange(10)
b.resize_(2,5)


tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])

In [32]:
b.unsqueeze_(2)
# b=b.view(2,5)
b.shape

torch.Size([2, 5, 1, 1, 1, 1, 1, 1])

In [32]:
import torch as t
a=t.arange(12).resize(2,2,3)
a[:,1,:]

tensor([[ 3,  4,  5],
        [ 9, 10, 11]])

In [25]:
a.gather(2,t.LongTensor([[[2,1],
                            [0,2],
                            ],[[0,1],[0,1]]]))

tensor([[[ 2,  1],
         [ 3,  5]],

        [[ 6,  7],
         [ 9, 10]]])

In [36]:
from torch.autograd import Variable
x=Variable(t.ones(4),requires_grad=True)
# x.requires_grad_(True)

In [40]:
y=x**2

In [41]:
z=y*2

In [45]:
z.sum().backward()

In [47]:
x.grad

tensor([4., 4., 4., 4.])

开始搞cnn

In [10]:
import torch as t
import torch.nn as nn
from torch.autograd import  Variable as V
import torch.nn.functional as F
import torch.optim as optim

In [11]:
# 划分训练与测试集
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0,test_size=0.2)

In [14]:
# 定义数据集
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
class TopicvecSet(Dataset):
    def __init__(self,train = True) -> None:
        super(TopicvecSet, self).__init__()
        if train == True:
            self.X = t.from_numpy(X_train.to_numpy()).float().view(-1,20) # 矩阵类型的
            self.X.requires_grad_(True)
            self.y =  t.from_numpy(y_train.to_numpy()).float().unsqueeze(1) # 必须是矩阵类型的
        else :
            self.X = t.from_numpy(X_test.to_numpy()).float().view(-1,20)
            self.X.requires_grad_(True)
            self.y =  t.from_numpy(y_test.to_numpy()).float().unsqueeze(1)
        self.len = self.X.shape[0]
    def __getitem__(self, index):
        return t.index_select(self.X,dim=0,index = t.LongTensor([index])), self.y[index]

    def __len__(self):
        return self.len
train_dataset = TopicvecSet(train=True) #取训练集
test_dataset = TopicvecSet(train=False)
train_loader = DataLoader(dataset =train_dataset, batch_size=2 , shuffle = False, drop_last=True) # 
test_loader = DataLoader(dataset= test_dataset, batch_size=1, shuffle= False)

In [271]:
t.from_numpy(X_train.to_numpy()).float().view(-1,20).shape

torch.Size([34513, 20])

In [15]:
# 定义cnn网络

class myCNN(nn.Module):
    def __init__(self) -> None:
        super(myCNN,self).__init__()
        # 第一层 卷积
        layer1 = nn.Sequential(
            nn.Conv1d(1,16,3,padding=1), #input 1, output 16, k_size 3 -->16, 20, 1
            # print(x.shape),
            nn.ReLU(True), #relu acitvate
            nn.MaxPool1d(2) #pool一下 --> 16, 10, 1
        )
        self.layer1 = layer1
        layer2 = nn.Sequential(
            nn.Conv1d(16,32,3,1,1), #--> 32, 10,1
            nn.ReLU(True),
            nn.MaxPool1d(2) #-->32, 5,1
        )
        self.layer2 = layer2
        layer3 = nn.Sequential(
            nn.Linear(160, 80), 
            nn.ReLU(True), 
            nn.Linear(80,40),
            nn.ReLU(True),
            nn.Linear(40,1)
        )
        self.layer3 = layer3

    def  forward(self, x):
        # print(x.shape)
        batch_size = x.size(0)
        conv1 = self.layer1(x)
        conv2 = self.layer2(conv1)
        output = t.sigmoid(self.layer3(conv2.view(batch_size,-1)))
        return output

In [17]:
# CPU上定义loss和优化器
mycnn = myCNN()
# device = t.device("cuda:0" if t.cuda.is_available() else "cpu")
# t.cuda.empty_cache()
# mycnn.to(device)
criterion = nn.BCELoss()
optimizer = optim.SGD(params=mycnn.parameters(), lr=0.01)

In [18]:
## 定义训练过程
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate (train_loader, 0):
        inputs, target = data
        # inputs, target = inputs.to(device), target.to(device)
        # forward + backward + update
        # forward 算loss
        outputs = mycnn(inputs)
        # print(outputs)
        loss = criterion(outputs, target)
        # print (epoch, loss.item())
        # backward 算梯度
        optimizer.zero_grad()
        loss.backward()
        # Update 更新参数
        optimizer.step()
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print(f'第{epoch+1}次epoch--->第{batch_idx+1}次batch--->损失为：{running_loss/300}')
            running_loss=0

In [36]:
# 测试过程
def test():
    correct = 0
    total = 0
    total_neg = 0### neg测试
    correct_neg = 0 ###neg测试
    with t.no_grad():
        for data in test_loader:
            inputs, labels = data
            # inputs, labels = inputs.to(device), labels.to(device)
            outputs = mycnn(inputs)
            outputs.apply_(lambda x:1.0 if outputs.item()>0.5 else 0.0)
            correct += (outputs == labels).sum().item()
            total += outputs.size(0)
            label_neg = labels[labels==0]
            outputs_neg = outputs[t.where(labels==0)]
            correct_neg += (outputs_neg == label_neg).sum().item()
            total_neg += outputs_neg.size(0)
    print('测试集上的准确率为：',correct/(1.0*total))
    print('负例正确率为：',correct_neg/(1.0*total_neg))

In [37]:
if __name__ == '__main__':
    for epoch in range(3):
        train(epoch)
        test()

第1次epoch--->第300次batch--->损失为：0.09865885697232443
第1次epoch--->第600次batch--->损失为：0.1330728701587456
第1次epoch--->第900次batch--->损失为：0.11698024263392047
第1次epoch--->第1200次batch--->损失为：0.12009250415091326
第1次epoch--->第1500次batch--->损失为：0.1074587033650217
第1次epoch--->第1800次batch--->损失为：0.11618712669316059
第1次epoch--->第2100次batch--->损失为：0.08002200182585512
第1次epoch--->第2400次batch--->损失为：0.10620969312420736
第1次epoch--->第2700次batch--->损失为：0.10541867592701844
第1次epoch--->第3000次batch--->损失为：0.10422074189021563
第1次epoch--->第3300次batch--->损失为：0.12409913616759392
第1次epoch--->第3600次batch--->损失为：0.08012452236085664
第1次epoch--->第3900次batch--->损失为：0.11353842438955325
第1次epoch--->第4200次batch--->损失为：0.08957111701194663
第1次epoch--->第4500次batch--->损失为：0.11532135141621741
第1次epoch--->第4800次batch--->损失为：0.11052495415594119
第1次epoch--->第5100次batch--->损失为：0.13113158114506707
第1次epoch--->第5400次batch--->损失为：0.12971899230324196
第1次epoch--->第5700次batch--->损失为：0.10150510703814992
第1次epoch--->第6000次batch--->损失为：0.121

In [23]:
label = t.tensor(y_test.to_numpy()).float()

In [35]:
# label[label==0]
label[t.where(label==0)]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 